# 1. Aspect Extraction & Sentiment Analysis

In [1]:
import pandas as pd
import numpy as np
from pyabsa.functional import ABSADatasetList
from pyabsa.functional import ATEPCCheckpointManager
import pickle

This script could only be used to manage NVIDIA GPUs,but no GPU found in your device


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from .optimizers import Adam, linear_decay
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/Users/xiaoxiangma/.local/lib/python3.7/site-packages/matplotlib/backend_bases.py:55: DeprecationWarning

Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: None
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets


In [2]:
res_reviews = pd.read_csv("Restaurant_Reviews.tsv", sep='\t')
review_list = res_reviews.Review.tolist()

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
try:
    with open('aspect_extractor.pkl', 'rb') as file:
        aspect_extractor = pickle.load(file)


except:  
    aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')
    with open('aspect_extractor.pkl', 'wb') as file:
        pickle.dump(aspect_extractor, file)

In [4]:
# atepc_result = aspect_extractor.extract_aspect(inference_source=res_reviews.Review.tolist(),
#                                                save_result=True,
#                                                print_result=True,  # print the result
#                                                pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
#                                                )

100%|██████████| 1188/1188 [00:03<00:00, 306.63it/s, preparing apc inference dataloader...]
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:359: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  lcf_cdm_vec = torch.tensor([f.lcf_cdm_vec for f in infer_features], dtype=torch.float32)
  0%|          | 0/10 [00:00<?, ?it/s, classifying aspect sentiments...]/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:407: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]
100%|██████████| 10/10 [04

The results of aspect term extraction have been saved in /Users/xiaoxiangma/Documents/GitHub/reviews_analysis/atepc_inference.result.json
Text: Wow . . . Loved this <place:Positive> .
Text: <Crust:Negative> is not good .
Text: Not tasty and the <texture:Negative> was just nasty .
Text: Stopped by during the late May bank holiday off Rick Steve recommendation and loved it .
Text: The <selection:Neutral> on the <menu:Neutral> was great and so were the <prices:Positive> .
Text: Now I am getting angry and I want my damn <pho:Negative> .
Text: Honeslty it didn ' t taste THAT fresh . )
Text: The <potatoes:Negative> were like rubber and you could tell they had been made up ahead of time being kept under a warmer .
Text: The <fries:Positive> were great too .
Text: A great <touch:Positive> .
Text: <Service:Positive> was very prompt .
Text: Would not go back .
Text: The <cashier:Negative> had no care what so ever on what I had to say it still ended up being wayyy overpriced .
Text: I tried the C

Text: I kept looking at the <time:Neutral> and it had soon become 35 minutes , yet still no <food:Negative> .
Text: I have been to very few places to eat that under no circumstances would I ever return to , and this tops the list .
Text: We started with the <tuna sashimi:Negative> which was brownish in <color:Negative> and obviously wasn ' t fresh .
Text: <Food:Negative> was below average .
Text: It sure does beat the <nachos:Negative> at the <movies:Neutral> but I would expect a little bit more coming from a restaurant .
Text: All in all , Ha Long Bay was a bit of a flop .
Text: The problem I have is that they charge $ 11 . 99 for a <sandwich:Negative> that is no bigger than a Subway <sub:Neutral> ( which offers better and more amount of <vegetables:Positive> ) .
Text: <Shrimp:Negative> - When I unwrapped it ( I live only 1 / 2 a mile from Brushfire ) it was literally ice cold .
Text: It lacked <flavor:Negative> , seemed undercooked , and dry .
Text: It really is impressive that the <

# 2. Prepare for clustering

In [7]:
import json
  
# Opening JSON file
f = open('atepc_inference.result.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
f.close()

In [64]:
main = []
for e, sentence in enumerate(data[:]):
    single = []
    if True:
        single.append(e)
        single.append(sentence['sentence'])
        single.append(sentence['tokens'])
        single.append(sentence['aspect'])
        single.append([i[0] for i in sentence['position']])
        single.append(sentence['sentiment'])
        main.append(single)
main = pd.DataFrame(main, columns =['sentence_id', 'sentence', 'tokens', 'aspect', 'aspect_loc', 'aspect_sentiment'])

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
main

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentence_id,sentence,tokens,aspect,aspect_loc,aspect_sentiment
0,0,Wow . . . Loved this place .,"[Wow, ., ., ., Loved, this, place, .]",[place],[7],[Positive]
1,1,Crust is not good .,"[Crust, is, not, good, .]",[Crust],[1],[Negative]
2,2,Not tasty and the texture was just nasty .,"[Not, tasty, and, the, texture, was, just, nas...",[texture],[5],[Negative]
3,3,Stopped by during the late May bank holiday of...,"[Stopped, by, during, the, late, May, bank, ho...",[],[],[]
4,4,The selection on the menu was great and so wer...,"[The, selection, on, the, menu, was, great, an...","[selection, menu, prices]","[2, 5, 12]","[Neutral, Neutral, Positive]"
...,...,...,...,...,...,...
995,995,I think food should have flavor and texture an...,"[I, think, food, should, have, flavor, and, te...","[food, flavor, texture]","[3, 6, 8]","[Neutral, Negative, Negative]"
996,996,Appetite instantly gone .,"[Appetite, instantly, gone, .]",[Appetite],[1],[Negative]
997,997,Overall I was not impressed and would not go b...,"[Overall, I, was, not, impressed, and, would, ...",[],[],[]
998,998,"The whole experience was underwhelming , and I...","[The, whole, experience, was, underwhelming, ,...","[experience, Sushi]","[3, 17]","[Negative, Neutral]"


In [67]:
aspects_loc = {}
for e, sentence in enumerate(data[:]):
    if sentence['aspect'] != []:
        for i in range(len(sentence['aspect'])):
            aspects_loc[(e,
                sentence['position'][i][0]
            )] = sentence['aspect'][i]
aspects_loc = pd.DataFrame.from_dict(aspects_loc, orient='index')
aspects_loc.columns = ['aspect']

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
aspects_loc

,aspect
"(0, 7)",place
"(1, 1)",Crust
"(2, 5)",texture
"(4, 2)",selection
"(4, 5)",menu
...,...
"(995, 8)",texture
"(996, 1)",Appetite
"(998, 3)",experience
"(998, 17)",Sushi


# 3. Clustering: Umap & HDBscan

In [69]:
import random
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan
import umap
import tensorflow as tf

import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer

from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

from tqdm.notebook import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C hea

In [70]:
all_intents = list(aspects_loc['aspect'])
len(all_intents)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1188

In [73]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model_use = hub.load(module_url)

# print(f"module {module_url} loaded")

In [74]:
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('all-distilroberta-v1')